In [7]:
from rdkit import Chem
from pathlib import Path
from rdkit.Chem.Draw import IPythonConsole

mols = []

for path in Path('rocs').glob('rocs_*.sdf'):
    experiment = path.name[len('rocs_'):-len('.sdf')]
    with Chem.SDMolSupplier(str(path)) as sdfh:
        mol: Chem.Mol
        for mol in sdfh:
            for key in ('id', 'ID', 'Catalog ID', 'idnumber'):
                if mol.HasProp(key):
                    catalog_name = mol.GetProp(key)
                    break
            else:
                print(mol.GetProp('_Name'), mol.GetPropsAsDict())
                raise Exception
            mol.SetProp('_Name', catalog_name)
            mol.SetProp('experiment', experiment) 
            mols.append( mol )

print(len(mols))

4000


In [ ]:
import pandas as pd
from fragmenstein import Laboratory, place_input_validator

pdbblock = Path('x0310_apo.pdb').read_text()

# Laboratory.replace_hits(pdbblock, mols, ,
#                         suffix='redocked')

Laboratory.Victor.work_path = '/data/outerhome/tmp/rocs'
lab = Laboratory(pdbblock=pdbblock, covalent_resi=None, run_plip=True)
selfies = pd.DataFrame([dict(name=hit.GetProp('_Name')+'_replaced',
                             hits=[hit],
                             smiles=Chem.MolToSmiles(hit)
                             ) for hit in mols])
replacements: pd.DataFrame = lab.place(place_input_validator(selfies), n_cores=120, timeout=600)
Laboratory.fix_intxns(replacements)
replacements['bleached_name'] = replacements['name']

In [12]:
replacements['name'] = replacements.hit_mols.apply(lambda ms: ms[0].GetProp('_Name') if ms and len(ms) and isinstance(ms[0], Chem.Mol) else '')
replacements.to_pickle(f'ROCS_scored.pkl.gz')

In [101]:
import plotly.io as pio
pio.renderers.default='iframe'  
import plotly.express as px
from fragmenstein.branding import divergent_colors

px.histogram(replacements, 'experiment', color='outcome', template='plotly_white',
             category_orders={'outcome': ['acceptable', 'too moved', 'crashed', 'timeout', 'too contored']},
             color_discrete_map={'acceptable': divergent_colors[3][0], 'too moved': divergent_colors[3][2], 'crashed': divergent_colors[3][1], 'timeout': divergent_colors[3][1], 'too contored': divergent_colors[3][1]},
             title='ROCS rescored with Fragmenstein')

/data/xchem-fragalysis/mferla/waconda/envs/compchem/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [56]:
vcs = replacements.hit_mols.apply(lambda ms: ms[0] if ms and len(ms) and isinstance(ms[0], Chem.Mol) else Chem.Mol())
for key in replacements.hit_mols[0][0].GetPropsAsDict():
    if key in ('id', 'smiles', 'Id'):
        continue
    if key == 'ROCS_ShapeQuery' or 'ROCS' not in key:
        replacements[key] = vcs.apply(lambda mol: mol.GetProp(key).strip() if mol.HasProp(key) else '')
    else:
        replacements[key] = vcs.apply(lambda mol: float(mol.GetProp(key).strip()) if mol.HasProp(key) else float('nan')).astype(float)

replacements = replacements.copy()

In [57]:
from collections import defaultdict
from Bio.SeqUtils import seq1

def narrate(row: pd.Series):
    grouped = defaultdict(list)
    for name, value in row.items():
        if not isinstance(name, tuple) or value == 0.:
            continue
        itxn_type, resn, resi = name
        grouped[itxn_type].append(seq1(resn, undef_code="X")+str(resi))
    narrative = f'info set:{row.experiment};'
    for itxn_type in sorted(grouped):
        narrative += f'{itxn_type}:{"+".join(grouped[itxn_type])}; '
    return narrative

replacements['rationale'] = 'info ' + replacements.apply(narrate, axis=1)
replacements.to_pickle(f'ROCS_scored.pkl.gz')
replacements.error.value_counts()

error
                                                                                                                                         3842
TimeoutError                                                                                                                              107
ValueError Bad Conformer Id                                                                                                                31
RuntimeError \n\nFile: /home/benchmark/rosetta/source/src/core/conformation/util.cc:1512\n[ ERROR ] UtilityExitException\nERROR: \n\n      17
AssertionError Molecule too horrendous to load.                                                                                             3
Name: count, dtype: int64

In [72]:
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors

from typing import List, Dict, Any, Optional
import operator, os, re, logging, random, time, argparse, string, itertools, json, contextlib, requests
from warnings import warn
import pandas as pd
import pandera.typing as pdt
from pandarallel import pandarallel
from smallworld_api import SmallWorld


from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import AllChem, Draw, PandasTools, BRICS
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFingerprintGenerator as rdfpg
from rdkit.Chem.rdfiltercatalog import FilterCatalogParams, FilterCatalog, FilterCatalogEntry

# ----- Scoring -----------------------------------
params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
catalog = FilterCatalog(params)

def get_pains(mol) -> List[str]:
    with contextlib.suppress(Exception):
        entry: FilterCatalogEntry
        if not isinstance(mol, Chem.Mol) or mol.GetNumHeavyAtoms() == 0:
            return []
        AllChem.SanitizeMol(mol)
        return [entry.GetDescription() for entry in catalog.GetMatches(mol)]

class GetRowSimilarity:
    def __init__(self, hits):
        self.fpgen = rdfpg.GetRDKitFPGenerator()
        self.hit2fp = {h.GetProp('_Name'): self.fpgen.GetFingerprint(h) for h in hits}

    def __call__(self, row: pd.Series):
        with contextlib.suppress(cli_default_settings['supressed_exceptions']):
            if not isinstance(row.minimized_mol, Chem.Mol):
                return float('nan')
            elif isinstance(row.hit_names, str):
                hit_names = row.hit_names.split(',')
            elif isinstance(row.hit_names, list):
                hit_names = row.hit_names
            else:
                return float('nan')
            fp = self.fpgen.GetFingerprint(AllChem.RemoveHs(row.minimized_mol))
            return max([DataStructs.TanimotoSimilarity(fp, self.hit2fp[name]) for name in hit_names])
        return float('nan')


class HitIntxnTallier:

    def __init__(self, hit_replacements):
        self.slim_hits = self.slim_down(hit_replacements)

    def slim_down(self, hit_replacements):
        # the bleaching was fixed cf bleached_name
        # hit_replacements['new_name'] = hit_replacements.name.str.replace('-', '_')
        # undoing bleaching
        hit_replacements['new_name'] = hit_replacements.hit_mols.apply(lambda ms: ms[0].GetProp('_Name'))
        columns = [c for c in hit_replacements.columns if isinstance(c, tuple)]
        return hit_replacements.set_index('new_name')[columns].fillna(0).copy()

    def __call__(self, row: pd.Series):
        with contextlib.suppress(cli_default_settings['supressed_exceptions']):
            if not isinstance(row.minimized_mol, Chem.Mol) or isinstance(row.hit_names, float):
                return float('nan'), float('nan')
            present_tally = 0
            absent_tally = 0
            for hit_name in list(row.hit_names):
                if hit_name not in self.slim_hits.index:
                    raise Exception('Name' + hit_name)
                hit_row = self.slim_hits.loc[hit_name]
                for intxn_name, hit_value in hit_row.items():
                    if not hit_value:
                        continue
                    elif intxn_name not in row.index:
                        absent_tally += 1 if intxn_name[0] != 'hydroph_interaction' else 0.5
                    elif row[intxn_name]:
                        absent_tally += 1 if intxn_name[0] != 'hydroph_interaction' else 0.5
                    else:
                        present_tally += 1 if intxn_name[0] != 'hydroph_interaction' else 0.5
            return present_tally, absent_tally
        return float('nan'), float('nan')


class UniquenessMeter:
    def __init__(self, tallies, intxn_names, k=0.5):
        self.tallies = tallies
        self.intxn_names = intxn_names
        self.k = k

    def __call__(self, row):
        with contextlib.suppress(cli_default_settings['supressed_exceptions']):
            return sum([(row[name] / self.tallies[name]) ** self.k for name in self.intxn_names if
                        row[name] and self.tallies[name]])
        return float('nan')

    def tally_interactions(self, row):
        return sum([row[c] if self.intxn_names[0] != 'hydroph_interaction' else row[c] * 0.5 for c in self.intxn_names])


class PenaltyMeter:
    def __init__(self, weights, nan_penalty=10):
        self.weights = weights
        self.nan_penalty = nan_penalty

    def __call__(self, row):
        with contextlib.suppress(cli_default_settings['supressed_exceptions']):
            penalty = 0
            if row.outcome != 'acceptable':
                return float('inf')
            for col, w in self.weights.items():
                if col not in row.index:
                    warn(f'{col} column is missing from df')
                    continue
                penalty += row[col] * w if str(row[col]) != 'nan' else self.nan_penalty
            return penalty
        return float('nan')


def butina_cluster(mol_list, cutoff=0.35):
    # https://github.com/PatWalters/workshop/blob/master/clustering/taylor_butina.ipynb
    fp_list = [rdmd.GetMorganFingerprintAsBitVect(AllChem.RemoveAllHs(m), 3, nBits=2048) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in range(1, nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        dists.extend([1 - x for x in sims])
    mol_clusters = Butina.ClusterData(dists, nfps, cutoff, isDistData=True)
    cluster_id_list = [0] * nfps
    for idx, cluster in enumerate(mol_clusters, 1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list

def UFF_Gibbs(mol):
    # free energy cost of bound conformer
    if not isinstance(mol, Chem.Mol) or mol.GetNumHeavyAtoms() == 0:
        return float('nan')
    with contextlib.suppress(Exception):
        AllChem.SanitizeMol(mol)
        # this is actually UFF
        copy = Chem.Mol(mol)
        return Monster.MMFF_score(None, mol, True)
    return float('nan')

Laboratory.fix_intxns(replacements)
df = replacements
intxn_names = [c for c in df.columns if isinstance(c, tuple)]
tallies = df[intxn_names].sum()
ratioed = UniquenessMeter(tallies, intxn_names, k=0.5)
df['interaction_uniqueness_metric'] = df.apply(ratioed, axis=1)
df['N_interactions'] = df.apply(ratioed.tally_interactions, axis=1)
df['PAINSes'] = df.minimized_mol.apply(get_pains)
df['N_PAINS'] = df.PAINSes.apply(len)
df['UFF_Gibbs'] = df.minimized_mol.apply(UFF_Gibbs)
df['strain_per_HA'] = df.UFF_Gibbs / (df.N_HA + 0.0001)

weights = {"N_rotatable_bonds": 1.5,
             "\u2206\u2206G": 1,
               "ROCS_Rank": 3/100,
             "interaction_uniqueness_metric": -3,
             #"N_unconstrained_atoms": 0.2,
             #"N_constrained_atoms": -0.1,
             "N_interactions": -2,
             "N_PAINS": 7,
             "strain_per_HA": 0.3}

penalize = PenaltyMeter(weights)
df['ad_hoc_penalty'] = df.apply(penalize, axis=1)
with contextlib.suppress(Exception):
    m = df.minimized_mol.apply(lambda m: m if isinstance(m, Chem.Mol) else Chem.Mol())
    df['cluster'] = butina_cluster(m.to_list())

In [78]:
from scipy.cluster.vq import kmeans, vq
# number of clusters sought
k = 20

intxn_cols = [c for c in df.columns if isinstance(c, tuple)]
data_for_clustering = df.loc[df.outcome == 'acceptable'][intxn_cols].fillna(0).copy()
tallies = data_for_clustering.sum().to_dict()
data_for_clustering = data_for_clustering.apply(lambda col: col / tallies[col.name],axis=0).fillna(0)
centroid, variance = kmeans(data_for_clustering.values, k)
labels, _ = vq(data_for_clustering.values, centroid)

# list to series first for the correct indices:
data_for_clustering['cluster_label']: pdt.Series[int] = labels
df['cluster_label']: pdt.Series[float] = data_for_clustering.cluster_label
df['cluster_label']: pdt.Series[int] = df['cluster_label'].fillna(-1).astype(int)

In [110]:
from collections import defaultdict
from Bio.SeqUtils import seq1
rank = defaultdict(int)

def r(c):
    rank[c] += 1
    return rank[c]

# 'cluster' is Similarity cluster
df = df.loc[df.outcome == 'acceptable'].sort_values('ad_hoc_penalty').drop_duplicates('cluster').reset_index(drop=True)
df['cluster_rank'] = df['cluster_label'].apply(r)
df = df.sort_values(['cluster_rank', 'ad_hoc_penalty']).reset_index(drop=True).copy()

In [111]:
replacements = df

In [116]:
from gist_import import GistImporter

store = GistImporter.from_github('https://raw.githubusercontent.com/matteoferla/Fragment-hit-follow-up-chemistry/main/fragment_elaboration_scripts/enamine_store.py')\
                    .to_module()

In [124]:

get_price(replacements.name[0])

4.32

In [ ]:
import time
#costs = {}

for code in replacements.loc[(replacements['∆∆G'] < 0.) & (replacements['comRMSD'] < 2.)].sort_values('cluster_rank').name.to_list():
    if code in costs:
        print(code, costs[code])
    elif 'Z' in code or 'PV' in code:
        costs[code] = store.get_price(code, catalogue=store.StoreCatalog.REALDB, currency=store.StoreCurrency.USD)
        print(code, costs[code])
        time.sleep(10)
    elif 'EN' in code:
        costs[code] =store.get_price(code, catalogue=store.StoreCatalog.BB, currency=store.StoreCurrency.USD)
        print(code, costs[code])
        time.sleep(10)
    else:
        raise ValueError

EN300-26666438 4.32
Z45507458 82.1
EN300-27782356 13.2
EN300-2908013 1.88
Z1078498278 82.1
EN300-7489981 5.7
Z52591645 82.1
Z86302940 73.9
Z55915292 73.9
Z54409144 82.1
EN300-28298106 9.1
Z2739892545 82.1
Z55491204 73.9
Z94861526 73.9
Z31120760 82.1
Z1766224104 82.1
Z1716137365 82.1
EN300-7424825 16.1
EN300-365277 4.9
Z1764946926 82.1
EN300-1723691 4.36
Z2057224744 104.0
EN300-1618086 3.76
EN300-359480 23.76
EN300-277778 4.58
Z55423090 82.1
Z4967399033 148.6
Z729056044 73.9
EN300-37443701 6.88
Z99095183 82.1
Z2306626585 82.1
EN300-263007 6.3
Z5346215097 104.0
Z2394925055 82.1
EN300-45481960 3.72
Z56881412 73.9
EN300-26666437 6.22
EN300-107669 3.36
Z3714214241 82.1
Z2057627275 104.0
EN300-6511692 9.1
Z8820654813 104.0
Z910841134 82.1
Z2060926323 104.0
Z4351154733 104.0
EN300-206684 3.64
Z1082767906 73.9
EN300-206714 0.5
Z1624949930 82.1
Z3044709108 82.1
EN300-219365 4.24
EN300-107837 3.36
Z98003105 82.1
Z3542299488 104.0
Z2001735784 82.1
Z238750748 73.9
Z168123244 73.9
EN300-28298107 9.

In [107]:
from gist_import import GistImporter

# fu for fragalysis upload
fmodule = GistImporter.from_github('https://raw.githubusercontent.com/matteoferla/Fragment-hit-follow-up-chemistry/main/fragment_elaboration_scripts/prep_fragalysis.py')
prep = fmodule['prep']
generate_header = fmodule['generate_header']
floatify_columns = fmodule['floatify_columns']

In [114]:
wanted_key_types = {'rationale': str, 
                    'experiment': str,
                    'cluster_label': int,
               'cluster_rank': int,
               'N_interactions': int, 
               '∆∆G': float, 
                    'comRMSD': float,
                    #'N_rotatable_bonds': int, 
                    'ROCS_TanimotoCombo': float,
                     'ROCS_ShapeTanimoto': float,
                     'ROCS_ColorTanimoto': float,
                   }

for k, ktype in wanted_key_types.items():
    replacements[k] = replacements[k].astype(ktype)

wanted_keys = list(wanted_key_types)

def clean_names(names):
    return ','.join(['x0446_0A','x1080_0A','x0929_0A','x0812_0A','x0719_0A','x1140_0A','x0528_0A','x0310_0A'])

replacements['ref_mols'] = replacements.hit_names.apply(clean_names)

method_name = 'A71-ROCS-iter2'
header: Chem.Mol = generate_header(method=method_name,
                         ref_url='https://github.com/matteoferla/EV-A71-2A-elaborations',
                         submitter_name='Matteo Ferla',
                         submitter_email='matteo.ferla@stats.ox.ac.uk',
                         submitter_institution='University of Oxford',
                         extras=dict(zip(wanted_keys, wanted_keys))
                                  )
                                   
prep(replacements.loc[(replacements['∆∆G'] < 0.) & (replacements['comRMSD'] < 2.)].sort_values('cluster_rank'), 
     header, mol_col='minimized_mol', 
     name_col='name',
     outfile=f'{method_name}.sdf',
     ref_pdb_name='x0310_0A',
     extras=wanted_keys
    )